In [4]:
!git clone https://github.com/Atharva-Malode/ML-Bootcamp.git

Cloning into 'ML-Bootcamp'...
remote: Enumerating objects: 425, done.
remote: Counting objects: 100% (232/232), done.
remote: Compressing objects: 100% (160/160), done.
remote: Total 425 (delta 100), reused 152 (delta 65), pack-reused 193
Receiving objects: 100% (425/425), 8.68 MiB | 16.35 MiB/s, done.
Resolving deltas: 100% (141/141), done.


In [3]:
!pip install wordcloud

In [2]:
import numpy as np
import pandas as pd
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [9]:
df = pd.read_csv("/content/ML-Bootcamp/Week-3/Day-1/Dataset/SMSSpamCollection.txt", sep="\t", names = ['label','text'])
df

,label,text
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will ü b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...


In [7]:
df.shape

(5572, 2)

In [10]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [12]:
sn = SnowballStemmer("english")
stop = set(stopwords.words("english"))

In [22]:
def clean_text(sms):
  sms = sms.lower()
  sms = re.sub("[a^-z]",' ',sms)
  sms = nltk.word_tokenize(sms)
  sms = [t for t in sms if len(t)>1]
  sms = [sn.stem(word)for word in sms if word not in stop]
  sms = ' '.join(sms)


  return sms

In [23]:
clean_text("Get this stuff for free ? just Rs.10 ")

'.10'

In [30]:
from wordcloud import WordCloud

In [35]:
def wordCloud(data):
  words = ' '.join(data)
  wc = WordCloud(background_color='white')
  wc = wc.generate(words)
  plt.figure(figsize = (10,8))
  plt.imshow(wc)
  plt.axis("off")
  plt.show()

In [37]:
def wordCloud(data):
  words = ' '.join(data)
  wc = WordCloud(background_color='white')
  wc = wc.generate(words)
  plt.figure(figsize = (10,8))
  plt.imshow(wc)
  plt.axis("off")
  plt.show()